# Python + Pandas is love

L'idéal pour une rapport de données, la validation d'une intuition, vite fait, sur un coin de table, en mode oneshot / quick&dirty ? ~~Excel~~ [Pandas](https://pandas.pydata.org/) !

Nous allons suivre la génération d'un rapport à partir d'un _petit_ dump de données, ce qui va nous permettre de faire une visite guidée des fonctionnalités de pandas. Ce ne sera pas exhaustif, mais devrait vous permettre d'y penser la prochaine fois. 


## La base


In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

### Serie

Une série, est une séquence de valeurs, de type homogène (toutes du même type : String, entiers, flotants, dates ... ), __ avec un index __

In [2]:
s = pd.Series([5,4,3,2,1,2,3,4,5])
s_datetimes = pd.date_range(start="2017-01-01", periods=86400, freq="2H").to_series()

display(s.head())
display(s_datetimes.head())

0    5
1    4
2    3
3    2
4    1
dtype: int64

2017-01-01 00:00:00   2017-01-01 00:00:00
2017-01-01 02:00:00   2017-01-01 02:00:00
2017-01-01 04:00:00   2017-01-01 04:00:00
2017-01-01 06:00:00   2017-01-01 06:00:00
2017-01-01 08:00:00   2017-01-01 08:00:00
Freq: 2H, dtype: datetime64[ns]

## Dataframe

Un Dataframe, c'est un tableau : 
- un ensemble de Series, nommées ( les colonnes)
- qui partagent un index


In [3]:
data = pd.DataFrame({"time": s_datetimes[:120].values,  # get rid of the index
                     "value": np.random.random(120),
                     "category": pd.Categorical(list("abcdef" * 20))})

print(data.info(()))
display(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 3 columns):
category    120 non-null category
time        120 non-null datetime64[ns]
value       120 non-null float64
dtypes: category(1), datetime64[ns](1), float64(1)
memory usage: 2.3 KB
None


,category,time,value
0,a,2017-01-01 00:00:00,0.733525
1,b,2017-01-01 02:00:00,0.222922
2,c,2017-01-01 04:00:00,0.316829
3,d,2017-01-01 06:00:00,0.422121
4,e,2017-01-01 08:00:00,0.510658


### Import/Export

L'idée étant évidemment d'importer nos données ici, et pas de les générer. Pandas supporte à peu près tout les fichiers formats de données courants :
- csv 
- excel 
- hdf5 
- json/msgpack/parquet

_mais aussi_  on peut créer un dataframe directement depuis une requête SQL, ou même Big Query.

Nous allons ici utiliser la lingua franca de la data ( rappel du contexte : on est sale, on est rapide, on est pas sexys ...) : le csv. 

In [27]:
clients = pd.read_csv("client_files.csv", delimiter=";", index_col=0, parse_dates=[1])
display(clients.head(4))
events = pd.read_csv("events.csv", delimiter=";", index_col=0, parse_dates=True)
#display(events.info())
print("events is row x cols", events.shape)
display(events.head(4))

,date_created,product,first_name,last_name,dob
uuid,,,,,
cdc4184a-1ea9-4bbb-b73a-3b43ffec13a2,2017-01-01 00:57:10,CARTE_CREDIT,Arthur,Chevallier,2018-02-19 14:16:18
d5f69c02-9db3-4fa9-9c56-9005ebe57384,2017-01-01 01:03:54,ASSURANCE_A,Jérôme,Tessier,2018-03-13 21:33:39
05994cd0-4804-43c8-af0b-d9cb17c924dc,2017-01-01 01:37:00,ASSURANCE_A,Gérard,Merle,2018-03-06 13:44:38
b11380bd-beae-4502-a97c-a6a2562bb58c,2017-01-01 01:53:57,ASSURANCE_A,Adrien,Vallet,2018-02-24 12:02:46


events is row x cols (23452, 4)


,client_uuid,type,doc_type,status
timestamp,,,,
2017-01-01 00:59:11,cdc4184a-1ea9-4bbb-b73a-3b43ffec13a2,DOC,RIB,True
2017-01-01 01:03:29,cdc4184a-1ea9-4bbb-b73a-3b43ffec13a2,DOC,TAX_NOTICE,True
2017-01-01 01:04:09,cdc4184a-1ea9-4bbb-b73a-3b43ffec13a2,SIGNATURE,NaN,True
2017-01-01 01:05:27,d5f69c02-9db3-4fa9-9c56-9005ebe57384,DOC,TAX_NOTICE,False
